# This code is to try and learn prompt engineering using Llama 3

In [46]:
import langchain
import ollama
from langchain_community.llms import Ollama
from crewai import Agent, Task, Crew, Process

In [21]:
model = Ollama(model="llama3")

# Basic prompt enegineering techniques

In [57]:
email = ''' 
subject: Urgent, please sign the approval
Text: dear Reza,
        Hope this email finds you well, we need your signature to proceed with the deployment of the PFR model.
    please reach out if any questions.
    Sincerely,
    AI platform team
    '''

In [66]:
system_content = ''' 
    You are an AI assistant whoes job is to accurately classify emails in an honest and upfront way. 
    there are only 3 categories for importance rating: 'important', 'casual', 'spam'.
    You need to be careful not to let any spam email to pass through, so should be inclined to rating suspicious emails as spam. 
    reply with the final category and the reason for that.
    '''
user_content = f"what is the category of this email: {email}"

In [83]:
response = ollama.chat(model='llama3', 
                       messages=[
                           {'role': 'system', 'content': system_content},
                           {'role': 'user', 'content': user_content}
                                 ],
                        stream=False,
                        options = {'temperature' : 0})

In [84]:
print(response['message']['content'])

I would categorize this email as "important".

The reason for this is that the subject line indicates a sense of urgency and importance, and the text mentions a specific action (signing an approval) that needs to be taken. The tone of the email is professional and polite, which suggests that it's not spam or casual in nature. Additionally, the email appears to be from a legitimate team (AI platform team), which further supports its categorization as important.


In [91]:
response

{'model': 'llama3',
 'created_at': '2024-04-25T08:36:14.756955Z',
 'message': {'role': 'assistant',
  'content': 'I would categorize this email as "important".\n\nThe reason for this is that the subject line indicates a sense of urgency and importance, and the text mentions a specific action (signing an approval) that needs to be taken. The tone of the email is professional and polite, which suggests that it\'s not spam or casual in nature. Additionally, the email appears to be from a legitimate team (AI platform team), which further supports its categorization as important.'},
 'done': True,
 'total_duration': 4605200875,
 'load_duration': 2931417,
 'prompt_eval_duration': 1043773000,
 'eval_count': 94,
 'eval_duration': 3551423000}

In [93]:
# alternative way to print the outputs
stream = ollama.chat(model='llama3', 
                       messages=[
                           {'role': 'system', 'content': system_content},
                           {'role': 'user', 'content': user_content}
                                 ],
                        stream=True,
                        options = {'temperature' : 0})

assistant_message = ''
for chunk in stream:
    assistant_message += chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)

I would categorize this email as "important".

The reason for this is that the subject line indicates a sense of urgency and importance, and the text mentions a specific action (signing an approval) that needs to be taken. The tone of the email is professional and polite, which suggests that it's not spam or casual in nature. Additionally, the email appears to be from a legitimate team (AI platform team), which further supports its categorization as important.

In [87]:
# Initializing an empty list for storing the chat messages and setting up the initial system message
chat_messages = []
system_message='You are a helpful assistant.'

# Defining a function to create new messages with specified roles ('user' or 'assistant')
def create_message(message, role):
  return {
    'role': role,
    'content': message
  }

# Starting the main conversation loop
def chat():
  # Calling the ollama API to get the assistant response
  ollama_response = ollama.chat(model='llama3', stream=True, messages=chat_messages)

  # Preparing the assistant message by concatenating all received chunks from the API
  assistant_message = ''
  for chunk in ollama_response:
    assistant_message += chunk['message']['content']
    print(chunk['message']['content'], end='', flush=True)
    
  # Adding the finalized assistant message to the chat log
  chat_messages.append(create_message(assistant_message, 'assistant'))

# Function for asking questions - appending user messages to the chat logs before starting the `chat()` function
def ask(message):
  chat_messages.append(
    create_message(message, 'user')
  )
  print(f'\n\n--{message}--\n\n')
  chat()

In [88]:
ask('Please list the 20 largest cities in the world.')



--Please list the 20 largest cities in the world.--


Here are the 20 largest cities in the world, ranked by population:

1. **Tokyo, Japan**: 38,140,000
2. **Delhi, India**: 29,280,000
3. **Shanghai, China**: 24,160,000
4. **Mumbai, India**: 21,380,000
5. **São Paulo, Brazil**: 21,310,000
6. **Mexico City, Mexico**: 20,900,000
7. **Osaka, Japan**: 20,310,000
8. **Kolkata, India**: 18,740,000
9. **Dhaka, Bangladesh**: 18,540,000
10. **Beijing, China**: 17,520,000
11. **Lagos, Nigeria**: 16,770,000
12. **Buenos Aires, Argentina**: 16,430,000
13. **Cairo, Egypt**: 15,850,000
14. **Los Angeles, USA**: 15,620,000
15. **Calcutta, India**: 15,510,000
16. **Chicago, USA**: 15,440,000
17. **Bangkok, Thailand**: 14,920,000
18. **Nairobi, Kenya**: 14,550,000
19. **Istanbul, Turkey**: 14,420,000
20. **Kuala Lumpur, Malaysia**: 14,330,000

Note: Population figures are for the urban agglomeration (city proper) and are based on latest available estimates from reliable sources such as city governme

# Email Classification using agents

In [69]:
classifier = Agent(
    role = "email classifier",
    goal = "accurately classify emails based on their importance into one of the following 3 categories: important, casual, spam",
    backstory = "You are an AI assistant whoes job is to accurately classify emails in an honest and upfront way. \
        You need to be careful not to let any spam email to pass through, so should be inclined to rating suspicious emails as spam.",
    allow_delegation =  False,
    verbose = False,
    llm = model
)

In [70]:
responder = Agent(
    role = "email responder",
    goal = "Based on the importance rating of the email write a concise response. If the email is classified as spam, just ignore it.\
        if the email is classified as important, write a short formal response. \
        and if the email is classified as casual, just write a friendly response back",
    backstory = "You are an AI assistnat whoes job is to provide a short response to an email based on the importance rating.\
    The importance rating of the email is classified by the 'classifier' agent.",
    allow_delegation =  False,
    verbose  = False,
    llm = model
)

In [71]:
email_classfier = Task(
    description = f"classify the following email: {email}",
    agent = classifier,
    expected_output = "One of the following options: 'important', 'casual', 'spam'."
)

email_responder = Task(
    description = f"Write a response to the following email: {email}, based on the importance rating identified by the 'responder' agent.",
    agent = responder,
    expected_output = "A concise response based on the importance rating identified by the 'responder' agent."
)

crew = Crew(
    agents = [classifier],
    tasks = [email_classfier],
    process = Process.sequential,
    verbose = 2
)

2024-04-25 18:28:25,938 - 7921688256 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [72]:
output = crew.kickoff()
print(output)

 [DEBUG]: == Working Agent: email classifier
 [INFO]: == Starting Task: classify the following email:  subject: Urgent, please sign the approval
        Text: dear Reza,
        Hope this email finds you well, we need your signature to proceed with the deployment of the PFR model.
    please reach out if any questions.
    Sincerely,
    AI platform team
    
 [DEBUG]: == [email classifier] Task output: 'spam''<|eot_id|>assistant<|eot_id|><|eot_id|><|eot_id|><|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|start_header_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>


'spam''<|eot_id|>assistant<|eot_id|><|eot_id|><|eot_id|><|eot_id|> <

In [73]:
print(output)

'spam''<|eot_id|>assistant<|eot_id|><|eot_id|><|eot_id|><|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|> <|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|start_header_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>


### Notes

- You can see that this process takes a long time to complete
- Looks like direct usage of ollama in terminal could be faster
- You can use [Groq](https://wow.groq.com/) to accelerate the process significantly